In [26]:
%matplotlib inline
# Tyler Boudreau
# Trained on Miitary Aircraft Detection Dataset: https://www.kaggle.com/datasets/a2015003713/militaryaircraftdetectiondataset
from pathlib import Path
import os
import shutil
import numpy as np
import pandas as pd
from tqdm import tqdm
data_dir = Path("C:\\Users\\Tyler\\Desktop\\Orig_Data_Aircraft_Detection")

# read image dir
image_paths = []
annotation_paths = []
# collect image paths and annotations
for file_name in sorted(os.listdir(data_dir / 'dataset')):
    file_name = Path(file_name)
    if file_name.suffix == '.jpg':
        image_paths.append(data_dir / 'dataset' / file_name)
    if file_name.suffix == '.csv':
        annotation_paths.append(data_dir / 'dataset' / file_name)


In [27]:
#Exclude these aircraft as they are not in service or relevant.
exclude_classes = ['YF23', 'XB70', 'Vulcan']

class_names = [class_name for class_name in sorted(os.listdir(data_dir / 'crop')) if class_name not in exclude_classes]

class2idx = {class_name: i for i, class_name in enumerate(class_names)}

def filter_images_by_class(images_dir, exclude_classes):
    image_files = os.listdir(images_dir)
    filtered_images = [image for image in image_files if any(exclude_class not in image for exclude_class in exclude_classes)]
    return filtered_images

In [28]:
class2idx

{'A10': 0,
 'A400M': 1,
 'AG600': 2,
 'AV8B': 3,
 'B1': 4,
 'B2': 5,
 'B52': 6,
 'Be200': 7,
 'C130': 8,
 'C17': 9,
 'C2': 10,
 'C5': 11,
 'E2': 12,
 'E7': 13,
 'EF2000': 14,
 'F117': 15,
 'F14': 16,
 'F15': 17,
 'F16': 18,
 'F18': 19,
 'F22': 20,
 'F35': 21,
 'F4': 22,
 'J20': 23,
 'JAS39': 24,
 'MQ9': 25,
 'Mig31': 26,
 'Mirage2000': 27,
 'P3': 28,
 'RQ4': 29,
 'Rafale': 30,
 'SR71': 31,
 'Su34': 32,
 'Su57': 33,
 'Tornado': 34,
 'Tu160': 35,
 'Tu95': 36,
 'U2': 37,
 'US2': 38,
 'V22': 39}

In [29]:
def convert_bboxes_to_yolo_format(df: pd.DataFrame, class2idx: dict):
    df = df[df['class'].isin(class2idx.keys())]
    df['class'] = df['class'].apply(lambda x: class2idx[x]).values

    df['xmin'] = (df['xmin'] / df['width']).values
    df['ymin'] = (df['ymin'] / df['height']).values
    df['xmax'] = (df['xmax'] / df['width']).values
    df['ymax'] = (df['ymax'] / df['height']).values
    df['xc']   = (df['xmin'] + df['xmax']) / 2
    df['yc']   = (df['ymin'] + df['ymax']) / 2
    df['w']    = (df['xmax'] - df['xmin'])
    df['h']    = (df['ymax'] - df['ymin'])
    df.drop(
        ['filename', 'width', 'height', 'xmin', 'xmax', 'ymin', 'ymax'], 
        axis=1, 
        inplace=True
    )
    return df 

In [30]:
image_dir = data_dir / 'images'
label_dir = data_dir / 'labels'
os.makedirs(image_dir, exist_ok=True)
os.makedirs(label_dir, exist_ok=True)

In [ ]:
# create .txt annotations
for annotation_path in tqdm(annotation_paths):
    # get image_id
    image_id = annotation_path.parts[-1].split('.')[0]
    annotation_df = pd.read_csv(annotation_path)
    # transform to yolo format
    annotation_df = convert_bboxes_to_yolo_format(annotation_df, class2idx)
    # save to .txt resulting df
    with open(Path(label_dir) / f'{image_id}.txt', 'w') as f:
        f.write(annotation_df.to_string(header=False, index=False))

In [32]:
for image_path in image_paths:
    shutil.move(str(image_path), image_dir)

In [33]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set(font_scale=1.3)

In [36]:
image_paths = [Path(image_dir) / image_path for image_path in sorted(os.listdir(image_dir))]
label_paths = [Path(label_dir) / label_path for label_path in sorted(os.listdir(label_dir))]

In [39]:
from sklearn.model_selection import train_test_split

In [40]:
image_paths = [f'images/{image_path}' for image_path in sorted(os.listdir(image_dir))]

In [41]:
train_size = 0.95

train_image_paths, val_image_paths = train_test_split(
    image_paths, train_size=train_size, random_state=3573, shuffle=True)

In [42]:
# make train split
with open(data_dir / 'train_split.txt', 'w') as f:
    f.writelines(f'./{image_path}\n' for image_path in train_image_paths)

# make val split
with open(data_dir / 'val_split.txt', 'w') as f:
    f.writelines(f'./{image_path}\n' for image_path in val_image_paths)

In [43]:
data_yaml = "C:\\Users\\Tyler\\Desktop\\Orig_Data_Aircraft_Detection\\data\\MilitaryAircraft.yaml"

In [1]:
from ultralytics import YOLO

# Load a model
#model = YOLO("yolov8n.yaml")  # build a new model from scratch
model = YOLO("C:\\Users\\Tyler\\Downloads\\yolov8l.pt")  # load a pretrained model (recommended for training)

# Use the model
model.train(data="C:\\Users\\Tyler\\Desktop\\Orig_Data_Aircraft_Detection\\data\\MilitaryAircraft.yaml", epochs=350,lr0=0.00001,lrf=0.00001,optimizer='SGD')  # train the model

#path = model.export(format="onnx")  # export the model to ONNX format

Ultralytics YOLOv8.0.220 🚀 Python-3.11.6 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3080 Laptop GPU, 16384MiB)
engine\trainer: task=detect, mode=train, model=C:\Users\Tyler\Downloads\yolov8x.pt, data=C:\Users\Tyler\Desktop\Orig_Data_Aircraft_Detection\data\MilitaryAircraft.yaml, epochs=350, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train18, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, show=False, save_frames=False, save_txt=False, save_conf=F

Create a Git repo (`git init`) and commit (`git commit`).


DVCLive is detected and auto logging is enabled (run 'yolo settings dvc=False' to disable).
TensorBoard: Start with 'tensorboard --logdir runs\detect\train18', view at http://localhost:6006/
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅


train: Scanning C:\Users\Tyler\Desktop\Orig_Data_Aircraft_Detection\labels...:   0%|          | 0/11407 [00:00<?, ?it/s]Scanning C:\Users\Tyler\Desktop\Orig_Data_Aircraft_Detection\labels... 50 images, 0 backgrounds, 2 corrupt:   0%|          | 50/11407 [00:00<00:23, 483.88it/s]Scanning C:\Users\Tyler\Desktop\Orig_Data_Aircraft_Detection\labels... 106 images, 0 backgrounds, 4 corrupt:   1%|          | 106/11407 [00:00<00:21, 516.89it/s]Scanning C:\Users\Tyler\Desktop\Orig_Data_Aircraft_Detection\labels... 170 images, 0 backgrounds, 4 corrupt:   1%|▏         | 170/11407 [00:00<00:20, 555.33it/s]Scanning C:\Users\Tyler\Desktop\Orig_Data_Aircraft_Detection\labels... 237 images, 0 backgrounds, 7 corrupt:   2%|▏         | 237/11407 [00:00<00:18, 590.84it/s]Scanning C:\Users\Tyler\Desktop\Orig_Data_Aircraft_Detection\labels... 302 images, 0 backgrounds, 8 corrupt:   3%|▎         | 302/11407 [00:00<00:18, 597.52it/s]Scanning C:\Users\Tyler\Desktop\Orig_Data_Aircraft_Detection\labels... 364 im

train: WARNING ⚠️ C:\Users\Tyler\Desktop\Orig_Data_Aircraft_Detection\images\005a2bac1b0045cd972c75110c0f378b.jpg: ignoring corrupt image/label: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (3,) + inhomogeneous part.
train: WARNING ⚠️ C:\Users\Tyler\Desktop\Orig_Data_Aircraft_Detection\images\00ee2f5b81e8ff2f65cfa959cf30ef02.jpg: ignoring corrupt image/label: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (3,) + inhomogeneous part.
train: WARNING ⚠️ C:\Users\Tyler\Desktop\Orig_Data_Aircraft_Detection\images\01c1bbe086ccd2a86ff839287bb7d7e6.jpg: ignoring corrupt image/label: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (3,) + inhomogeneous part.
train: WARNING ⚠️ C:\Users\Tyler\Desktop\Orig_Data_Aircraft_Detection\images\01df70fc8d7

val: Scanning C:\Users\Tyler\Desktop\Orig_Data_Aircraft_Detection\labels...:   0%|          | 0/601 [00:00<?, ?it/s]Scanning C:\Users\Tyler\Desktop\Orig_Data_Aircraft_Detection\labels... 34 images, 0 backgrounds, 0 corrupt:   6%|▌         | 34/601 [00:00<00:01, 330.98it/s]Scanning C:\Users\Tyler\Desktop\Orig_Data_Aircraft_Detection\labels... 83 images, 0 backgrounds, 4 corrupt:  14%|█▍        | 83/601 [00:00<00:01, 412.17it/s]Scanning C:\Users\Tyler\Desktop\Orig_Data_Aircraft_Detection\labels... 126 images, 0 backgrounds, 4 corrupt:  21%|██        | 126/601 [00:00<00:01, 381.47it/s]Scanning C:\Users\Tyler\Desktop\Orig_Data_Aircraft_Detection\labels... 174 images, 0 backgrounds, 4 corrupt:  29%|██▉       | 174/601 [00:00<00:01, 390.83it/s]Scanning C:\Users\Tyler\Desktop\Orig_Data_Aircraft_Detection\labels... 215 images, 0 backgrounds, 5 corrupt:  36%|███▌      | 215/601 [00:00<00:00, 394.90it/s]Scanning C:\Users\Tyler\Desktop\Orig_Data_Aircraft_Detection\labels... 263 images, 0 backgrou

val: WARNING ⚠️ C:\Users\Tyler\Desktop\Orig_Data_Aircraft_Detection\images\1132a1dcba79950db0ef65ceade75add.jpg: ignoring corrupt image/label: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (3,) + inhomogeneous part.
val: WARNING ⚠️ C:\Users\Tyler\Desktop\Orig_Data_Aircraft_Detection\images\19bb9fe1a60bf3e834b9a43212cb9e90.jpg: ignoring corrupt image/label: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (3,) + inhomogeneous part.
val: WARNING ⚠️ C:\Users\Tyler\Desktop\Orig_Data_Aircraft_Detection\images\1abfb8ef640d7eede0c84b1a706a9949.jpg: ignoring corrupt image/label: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (3,) + inhomogeneous part.
val: WARNING ⚠️ C:\Users\Tyler\Desktop\Orig_Data_Aircraft_Detection\images\1b520bead342c72a25f

      1/350      14.9G     0.8568      4.804      1.259          7        640: 100%|██████████| 685/685 [10:12<00:00,  1.12it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:09<00:00,  1.94it/s]


                   all        579        853     0.0106      0.203     0.0131     0.0118

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/685 [00:00<?, ?it/s]

In [1]:
from ultralytics import YOLO
model = YOLO("C:\\Users\\Tyler\\AppData\\Local\\Programs\\Microsoft VS Code\\runs\\detect\\train8\\weights\\best.pt")


In [ ]:
import cv2
from sahi import AutoDetectionModel
from sahi.predict import get_prediction
from pathlib import Path
from ultralytics.utils.files import increment_path
from sahi.utils.yolov8 import download_yolov8l_model

def run(weights, source, view_img=False, save_img=False, exist_ok=False):
    # Check source path
    if not Path(source).exists():
        raise FileNotFoundError(f"Source path '{source}' does not exist.")

    # Download YOLOv8 model
    yolov8_model_path = weights
    download_yolov8l_model(yolov8_model_path)
    detection_model = AutoDetectionModel.from_pretrained(model_type='yolov8',
                                                         model_path=yolov8_model_path,
                                                         confidence_threshold=0.80,
                                                         device='cuda:0')
    # Video setup
    videocapture = cv2.VideoCapture(source)
    frame_width, frame_height = int(videocapture.get(3)), int(videocapture.get(4))
    fps, fourcc = int(videocapture.get(5)), cv2.VideoWriter_fourcc(*'mp4v')

    # Output setup
    save_dir = increment_path(Path('ultralytics_results_with_sahi') / 'exp', exist_ok)
    save_dir.mkdir(parents=True, exist_ok=True)
    video_writer = cv2.VideoWriter(str(save_dir / f'{Path(source).stem}.mp4'), fourcc, fps, (frame_width, frame_height))

    while videocapture.isOpened():
        success, frame = videocapture.read()
        if not success:
            break

        results = get_prediction(frame,
                                        detection_model,)
        object_prediction_list = results.object_prediction_list

        boxes_list = []
        clss_list = []
        for ind, _ in enumerate(object_prediction_list):
            boxes = object_prediction_list[ind].bbox.minx, object_prediction_list[ind].bbox.miny, \
                object_prediction_list[ind].bbox.maxx, object_prediction_list[ind].bbox.maxy
            clss = object_prediction_list[ind].category.name
            boxes_list.append(boxes)
            clss_list.append(clss)

        for box, cls in zip(boxes_list, clss_list):
            x1, y1, x2, y2 = box
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (56, 56, 255), 2)
            label = str(cls)
            t_size = cv2.getTextSize(label, 0, fontScale=0.8, thickness=1)[0]
            cv2.rectangle(frame, (int(x1), int(y1) - t_size[1] - 3), (int(x1) + t_size[0], int(y1) + 3), (56, 56, 255),
                          -1)
            cv2.putText(frame,
                        label, (int(x1), int(y1) - 2),
                        0,
                        0.6, [255, 255, 255],
                        thickness=1,
                        lineType=cv2.LINE_AA)

        if view_img:
            cv2.imshow(Path(source).stem, frame)
        if save_img:
            video_writer.write(frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    video_writer.release()
    videocapture.release()
    cv2.destroyAllWindows()


# Example usage in Jupyter Notebook
weights = "C:\\Users\\Tyler\\Desktop\\Orig_Data_Aircraft_Detection\\data\\runs\\detect\\train8\\weights\\best.pt"
source = "C:\\Users\\Tyler\\Downloads\\Aircraft_Video_Test.mp4"
run(weights, source, view_img=True, save_img=False, exist_ok=False)
